In [1]:
import os
from glob import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg

from pkgs.kitti_utils import *
from pkgs.kitti_detection_utils import *

from ultralytics import YOLO
from sklearn import linear_model
from sklearn import linear_model

In [7]:
#dataset 
DATA_PATH = r'./../dataset/2011_10_03_drive_0047_sync'

#image path
image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_02/data/*.png')))
#lidar path
lid_paths = sorted(glob(os.path.join(DATA_PATH, 'velodyne_points/data/*.bin')))
#GPS/IMU path
imu_paths = sorted(glob(os.path.join(DATA_PATH, r'oxts/data**/*.txt')))


In [13]:
cam_calib_file = './../dataset/2011_10_03_calib/calib_cam_to_cam.txt'
def cam_transformation(filepath):
    with open(filepath,'r') as f:
        calib = f.readlines()

    # get projection matrices (rectified left camera --> left camera (u,v,z))
    P_rect2_cam2 = np.array([float(x) for x in calib[25].strip().split(' ')[1:]]).reshape((3,4))

    # get rectified rotation matrices (left camera --> rectified left camera)
    R_ref0_rect2 = np.array([float(x) for x in calib[24].strip().split(' ')[1:]]).reshape((3, 3,))

    # add (0,0,0) translation and convert to homogeneous coordinates
    R_ref0_rect2 = np.insert(R_ref0_rect2, 3, values=[0,0,0], axis=0)
    R_ref0_rect2 = np.insert(R_ref0_rect2, 3, values=[0,0,0,1], axis=1)


    # get rigid transformation from Camera 0 (ref) to Camera 2
    R_2 = np.array([float(x) for x in calib[21].strip().split(' ')[1:]]).reshape((3,3))
    t_2 = np.array([float(x) for x in calib[22].strip().split(' ')[1:]]).reshape((3,1))

    # get cam0 to cam2 rigid body transformation in homogeneous coordinates
    T_ref0_ref2 = np.insert(np.hstack((R_2, t_2)), 3, values=[0,0,0,1], axis=0)
    
    return P_rect2_cam2, R_ref0_rect2, T_ref0_ref2

In [14]:
P_rect2_cam2,R_ref0_rect2,T_ref0_ref2 = cam_transformation(cam_calib_file)

In [16]:
def lid_transformation(filepath):

    T_velo_ref0 = get_rigid_transformation(filepath)
    return T_velo_ref0

In [18]:
lid_calib_file = './../dataset/2011_10_03_calib/calib_velo_to_cam.txt'
T_velo_ref0 = lid_transformation(lid_calib_file)

In [20]:
def imu_transformation(filepath):

    T_imu_velo = get_rigid_transformation(filepath)
    return T_imu_velo

In [22]:
imu_calib_file = './../dataset/2011_10_03_calib/calib_imu_to_velo.txt'
T_imu_velo = imu_transformation(imu_calib_file)

In [24]:
#detetction model
def detection_model(weights,classes):
    model = YOLO()
    model.classes = classes 
    return model


In [25]:
weights = "yolov8n.pt"
classes = [0, 1, 2, 3, 5, 7] # person, bicycle, car, motorcycle, bus, truck
model = detection_model(weights,classes)